In [1]:
# ============================================================
# train_model_v12_Bstrategy.py
#  - FE_v12 기반 Sequence → Next-Month Regression (B 전략)
#  - 무역 패턴 특화 dynamic feature 추가
# ============================================================

import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor


# ------------------------------------------------------------
# 0. PATH
# ------------------------------------------------------------
BASE_DIR = Path.cwd().resolve()
DATA_DIR = BASE_DIR.parents[1] / "data"

PAIR_FE_PATH  = "../preprocessing/pair_features_v12_segment_relaxed_spec.csv"
PIVOT_V6_PATH = DATA_DIR / "processed" / "pivot_value_v6.csv"


# ------------------------------------------------------------
# 1. LOAD DATA
# ------------------------------------------------------------
pair = pd.read_csv(PAIR_FE_PATH)
pivot = pd.read_csv(PIVOT_V6_PATH, index_col=0)

pivot.columns = pivot.columns.astype(int)
pivot = pivot.sort_index(axis=1).sort_index(axis=0)

items = set(pivot.index)
pair = pair[(pair["item_i"].isin(items)) & (pair["item_j"].isin(items))].reset_index(drop=True)

print("pair:", pair.shape)
print("pivot:", pivot.shape)


# ------------------------------------------------------------
# 2. STATIC FEATURES
# ------------------------------------------------------------
STATIC_FEATS = [
    "hs4_same", "hs3_same", "hs2_same", "hs_distance",
    "rolling_corr3", "rolling_corr6",
    "seg_max_corr", "seg_mean_pos_corr", "seg_high_cnt",
    "norm_diff_corr",
    "best_lag", "lag_corr",
    "scale_ratio", "iqr_ratio",
]


# ------------------------------------------------------------
# 3. BUILD TRAIN FRAME (B 전략)
# ------------------------------------------------------------
def build_training_frame(pair_df, pivot_df):
    rows = []
    t_cols = pivot_df.columns.to_list()
    n = len(t_cols)

    for row in pair_df.itertuples(index=False):
        A, B = row.item_i, row.item_j
        lag = int(row.best_lag)

        sA = pivot_df.loc[A].values.astype(float)
        sB = pivot_df.loc[B].values.astype(float)

        for t in range(max(lag, 2), n - 1):

            # baseline features
            b_t = sB[t]
            b_t_1 = sB[t - 1]
            a_t_lag = sA[t - lag]

            # --- B 전략 추가 feature ---
            a_t_lag_1 = sA[t - lag - 1]            # (1) lag 직전 신호
            delta_b_t = b_t - b_t_1                # (2) B 변화량
            delta_a_tlag = a_t_lag - a_t_lag_1     # (3) A leading 변화량
            ratio_ba = (b_t + 1) / (a_t_lag + 1)   # (4) smoothed ratio

            feat = {
                "item_i": A,
                "item_j": B,
                "t": t,
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "a_t_lag_1": a_t_lag_1,
                "delta_b_t": delta_b_t,
                "delta_a_tlag": delta_a_tlag,
                "ratio_ba": ratio_ba,
                "target": sB[t + 1],
            }

            # static FE 복사
            for c in STATIC_FEATS:
                feat[c] = getattr(row, c)

            rows.append(feat)

    return pd.DataFrame(rows)


train_df = build_training_frame(pair, pivot)
print("train_df:", train_df.shape)
print(train_df.head())


# ------------------------------------------------------------
# 4. TARGET TRANSFORM
# ------------------------------------------------------------
train_df["target_clip"] = train_df["target"].clip(lower=0)
train_df["target_log"] = np.log1p(train_df["target_clip"])

ranked = train_df["target_log"].rank(method="first")
train_df["target_bin"] = pd.qcut(ranked, q=5, labels=False, duplicates="drop").fillna(0).astype(int)

train_df["sample_weight"] = np.where(train_df["target_clip"] > 0, 1.0, 0.5)


# ------------------------------------------------------------
# 5. FEATURE LIST
# ------------------------------------------------------------
DYN_FEATS = [
    "b_t", "b_t_1", "a_t_lag",
    "a_t_lag_1",        # NEW
    "delta_b_t",        # NEW
    "delta_a_tlag",     # NEW
    "ratio_ba",         # NEW
]

FEATURES = DYN_FEATS + STATIC_FEATS


# ------------------------------------------------------------
# 6. LightGBM TRAINING
# ------------------------------------------------------------
def train_single_model(seed):
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

    oof = np.zeros(len(train_df))
    models = []

    for fold, (tr_idx, va_idx) in enumerate(kf.split(train_df, train_df["target_bin"])):
        tr = train_df.iloc[tr_idx]
        va = train_df.iloc[va_idx]

        model = LGBMRegressor(
            objective="regression",
            max_depth=4,
            num_leaves=31,
            subsample=0.7,
            colsample_bytree=0.7,
            min_child_samples=50,
            reg_alpha=0.2,
            reg_lambda=0.8,
            learning_rate=0.03,
            n_estimators=1500,
            random_state=seed,
            n_jobs=-1,
        )

        model.fit(
            tr[FEATURES],
            tr["target_log"],
            sample_weight=tr["sample_weight"],
            eval_set=[(va[FEATURES], va["target_log"])],
            eval_metric="rmse",
        )

        pred = np.expm1(model.predict(va[FEATURES]))
        pred = np.clip(pred, 0, None)

        oof[va_idx] = pred
        rmse = np.sqrt(mean_squared_error(va["target_clip"], pred))
        print(f"[Seed {seed}] Fold {fold} RMSE = {rmse:.4f}")

        models.append(model)

    rmse_all = np.sqrt(mean_squared_error(train_df["target_clip"], oof))
    print(f"🔥 Seed {seed} OOF RMSE = {rmse_all:.5f}")

    return models, oof


SEEDS = [42, 7, 2025, 314159, 999]
all_models, oof_sum = [], np.zeros(len(train_df))

for sd in SEEDS:
    print(f"\n===== Training SEED {sd} =====")
    models, oof = train_single_model(sd)
    all_models += models
    oof_sum += oof / len(SEEDS)

print("\n🎯 Final Ensemble OOF RMSE:",
      np.sqrt(mean_squared_error(train_df["target_clip"], oof_sum)))


# ------------------------------------------------------------
# 7. BUILD TEST FRAME
# ------------------------------------------------------------
def build_test_frame(pair_df, pivot_df):
    rows = []
    t_cols = pivot_df.columns.to_list()
    n = len(t_cols)

    for row in pair_df.itertuples(index=False):
        A, B = row.item_i, row.item_j
        lag = int(row.best_lag)

        sA = pivot_df.loc[A].values.astype(float)
        sB = pivot_df.loc[B].values.astype(float)

        if n - 1 - lag < 1:
            continue

        b_t = sB[-1]
        b_t_1 = sB[-2]
        a_t_lag = sA[-1 - lag]
        a_t_lag_1 = sA[-2 - lag]

        feat = {
            "item_i": A,
            "item_j": B,
            "b_t": b_t,
            "b_t_1": b_t_1,
            "a_t_lag": a_t_lag,
            "a_t_lag_1": a_t_lag_1,
            "delta_b_t": b_t - b_t_1,
            "delta_a_tlag": a_t_lag - a_t_lag_1,
            "ratio_ba": (b_t + 1) / (a_t_lag + 1),
        }

        for c in STATIC_FEATS:
            feat[c] = getattr(row, c)

        rows.append(feat)

    return pd.DataFrame(rows)


test_df = build_test_frame(pair, pivot)
pred = np.expm1(np.mean([m.predict(test_df[FEATURES]) for m in all_models], axis=0))
pred = np.clip(pred, 0, None)

submission = pd.DataFrame({
    "leading_item_id": test_df["item_i"],
    "following_item_id": test_df["item_j"],
    "value": pred.round().astype(int),
})

submission.to_csv("v12_Bstrategy_submission.csv", index=False)
print("\nSaved submission: v12_Bstrategy_submission.csv")


pair: (1097, 16)
pivot: (100, 43)
train_df: (42981, 25)
     item_i    item_j  t         b_t       b_t_1  a_t_lag  a_t_lag_1  \
0  AANGBULD  CCLHWFWF  4  16264199.0  26625256.0  14276.0   533478.0   
1  AANGBULD  CCLHWFWF  5  14379695.0  16264199.0  52347.0    14276.0   
2  AANGBULD  CCLHWFWF  6  24298240.0  14379695.0  53549.0    52347.0   
3  AANGBULD  CCLHWFWF  7  25017613.0  24298240.0      0.0    53549.0   
4  AANGBULD  CCLHWFWF  8  21530570.0  25017613.0  26997.0        0.0   

    delta_b_t  delta_a_tlag      ratio_ba  ...  rolling_corr3  rolling_corr6  \
0 -10361057.0     -519202.0  1.139189e+03  ...      -0.038619      -0.030293   
1  -1884504.0       38071.0  2.746943e+02  ...      -0.038619      -0.030293   
2   9918545.0        1202.0  4.537487e+02  ...      -0.038619      -0.030293   
3    719373.0      -53549.0  2.501761e+07  ...      -0.038619      -0.030293   
4  -3487043.0       26997.0  7.974876e+02  ...      -0.038619      -0.030293   

   seg_max_corr  seg_mean_pos_